In [1]:
import os
from mpi4py import MPI
import coqui

# CoQui computing environemnt: mpi handler and verbosity
mpi = coqui.MpiHandler()
coqui.set_verbosity(mpi, output_level=2, debug_level=0)

if os.path.exists("./thc.coulomb.h5"):
    os.remove("./thc.coulomb.h5")
if os.path.exists("./thc2.coulomb.h5"):
    os.remove("./thc.coulomb.h5")    

--------------------------------------------------------------------------
Ignoring value for oob_tcp_if_exclude on ccqlin065 (10.250.112.0/20: Did not find interface matching this subnet).
(You can safely ignore this message.)
--------------------------------------------------------------------------


# Coulomb Interaction

Given a single-particle basis set $\phi^{\textbf{k}}_a(\textbf{r})$ provided by `coqui.Mf`, the second step of initializing a many-body electronic structure simulation begins with computing the bare Coulomb integrals $V$. This process constructs the electron-electron interaction part of the second-quantizated Hamiltonian: 

$$
\hat{H}_{\mathrm{int}} = \frac{1}{2}\sum _{abcd}V^{\textbf{k}_1\textbf{k}_2\textbf{k}_3\textbf{k}_4} _{abcd} c^{\textbf{k}_1\dagger} _{a} c^{\textbf{k}_3\dagger} _{c} c^{\textbf{k}_4} _{d} c^{\textbf{k}_2} _{b}
$$
where the Coulomb matrix elements are defined as:
$$
V^{\textbf{k}_1\textbf{k}_2\textbf{k}_3\textbf{k}_4} _{abcd} = \int d\textbf{r} \int d\textbf{r}' \phi^{\textbf{k}_1*} _{a} (\textbf{r})\phi^{\textbf{k}_2} _{b}(\textbf{r})\frac{1}{|\textbf{r}-\textbf{r}'|}\phi^{\textbf{k}_3*} _{c}(\textbf{r}')\phi^{\textbf{k}_4} _{d}(\textbf{r}')
$$
This is the most general form of Coulomb Hamiltonian, capturing both *local* and *non-local* interactions among *all* electronic degrees of freedom. Here, the momentum transferred between the product basis 
$\rho^{\textbf{k}_{1}\textbf{k}_{2}}_{ab}(\textbf{r}) = \phi^{\textbf{k}_{1}*}_{a}(\textbf{r})\phi^{\textbf{k}_{2}}_{b}(\textbf{r})$
(also known as the pair densities) satisfies momentum conservation: 

$$
\textbf{k}_1 - \textbf{k}_2 + \textbf{G} = \textbf{k}_3 - \textbf{k}_4,
$$

where $\textbf{G}$ represents a reciprocal lattice vector of the system.

Despite the large dimensionality of the Coulomb tensor $V^{\mathbf{k}_1\mathbf{k}_2\mathbf{k}_3\mathbf{k}_4}_{abcd}$ - which scales cubically with $N_k$ and quartically with $N_{\mathrm{orb}}$ - the tensor is highly rank-deficient, due to the over-completeness of the product basis used to represent two-electron operators.

In realistic materials simulations, compressing the Coulomb tensor $V$ becomes essential to keep computational cost and memory usage manageable as the system size grows. The magic of CoQuí is that it offers systematic algorithms for efficiently constructing Coulomb Hamiltonians directly in compressed form. 

---
### 🔹 Tutorial Summary
In this tutorial, you will learn how to use CoQuí to:
- Construct the *ab initio* Coulomb Hamiltonian $V$ directly in the compressed THC representation
- Control the compression accuracy using user-defined parameters

## 1. Tensor Hypercontraction (THC)

The efficiency of CoQuí lies in its use of the **tensor hypercontraction (THC)** decomposition to represent the two-electron Coulomb integrals for a generic Bloch basis set: 

$$
V^{\textbf{k}_1\textbf{k}_2\textbf{k}_3\textbf{k}_4} _{abcd} \approx \sum^{N_{\mu}}_{\mu\nu}
X^{\textbf{k}_1*} _{\mu a} X^{\textbf{k}_2} _{\mu b}V^{\textbf{q}} _{\mu\nu} 
X^{\textbf{k}_3*} _{\nu c}X^{\textbf{k}_4} _{\nu d}
$$

Here, $\textbf{q} = \textbf{k}_1 - \textbf{k}_2 + \textbf{G} = \textbf{k}_4 - \textbf{k}_3 + \textbf{G}'$
is the momentum transfer, and the greek letters $\mu$, $\nu$ label the THC auxiliary basis functions. 

> 💡 **NOTE** 💡
> 
> - The THC representation fully separates the orbital indices $a$, $b$, $c$, and $d$ from each other in the Coulomb tensor (the same as for the crystal momenta). This not only reduces memory consumption but also enables low-scaling algorithms for the subseuqent electronic structure metehods. [[ref]]

---

## 1.1 Algorithm: Interpolative Separable Density Fitting (ISDF)

By default, CoQuí constructs the THC representation using the **interpolative separable density fitting (ISDF)** algorithm. 
In this approach, the pair density is approximated as 

$$
\rho^{\textbf{k}_{1}\textbf{k}_{2}}_{ab}(\textbf{r}) = \phi^{\textbf{k}_{1}*}_{a}(\textbf{r})\phi^{\textbf{k}_{2}}_{b}(\textbf{r}) \approx \sum_{\mu}^{N_{\mu}} \phi^{\textbf{k}_{1}*}_{a}(\textbf{r}_{\mu})\phi^{\textbf{k}_{2}}_{b}(\textbf{r}_{\mu}) \zeta^{\textbf{q}}_{\mu}(\textbf{r})
$$
where 
- $\textbf{q} = \textbf{k}_{1} - \textbf{k}_{2}$
- $\mathbf{r}_\mu$ are called **interpolating points**
- $\zeta^{\textbf{q}}_\mu(\mathbf{r})$ are the q-dependent **ISDF auxiliary basis functions**, also known as **interpolating vectors**


This low-rank approximation to the pair density naturally leads to the THC decomposition, where:

- **Collocation matrices**:

  $$
  X^{\mathbf{k}}_{\mu a} = \phi^{\mathbf{k}}_a(\mathbf{r}_\mu)
  $$

- **Projected Coulomb operator**:

  $$
  V^{\mathbf{q}}_{\mu \nu} = 
  \int d\mathbf{r} \int d\mathbf{r}' \,
  \zeta^{\mathbf{q} *}_\mu(\mathbf{r}) \, 
  \frac{1}{|\mathbf{r} - \mathbf{r}'|} \, 
  \zeta^{\mathbf{q}}_\nu(\mathbf{r}')
  $$

Here, $V^{\mathbf{q}}_{\mu \nu}$ represents the projection of the Coulomb kernel onto the auxiliary basis functions $\zeta^{\mathbf{q}}_\mu(\mathbf{r})$.

> 🧠 **Note:** ISDF is applicable to general basis functions, including both Kohn–Sham orbitals and Gaussian-type orbitals.  
> The only assumption in CoQuí is that the basis functions can be efficiently represented on a uniform real-space grid.

> 💡 **IMPORTANT** 💡
> 
> - The accuracy of the THC decomposition is entirely controlled by the number of ISDF auxiliary basis functions $N_{\mu}$.
> - On the other hand, $N_{\mu}$ directly impacts the cost of subsequent many-body electronic structure calculations.
> - In practice, $N_\mu = \alpha \cdot N_{\mathrm{orb}}$ with $\alpha \sim 5$-$20$ is typically sufficient, thanks to the low-rank structure of the full Coulomb tensor $V$!

## 🧱 1.2 `ThcCoulomb` Class

In CoQuí, the computation and access to the THC-decomposed Coulomb Hamiltonian is handled through the `ThcCoulomb` class. 
This class encapsulates the construction of the THC representation introduced in Section 1.1 and provides interfaces to the resulting tensors used in followup many-body calculations.

A standard usage looks like:
```python
# Mf for the target system
mf_params = {
    "prefix": "si", 
    "outdir": "data/qe_inputs/si/out", 
    "filetype": "h5"
}
mf = coqui.make_mf(mpi, mf_params, "qe")

# Construct ThcCoulomb and compute the THC integrals during initialization
thc_params = {
    "init": True
    "storage": "incore",
    "ecut": 40,
    "thresh": 1e-2
}
thc = coqui.make_thc_coulomb(mf, thc_params)
```
where the `ThcCoulomb` object is created via the function: 

```python
coqui.make_thc_coulomb(mf: coqui.Mf, thc_params: dict) -> coqui.ThcCoulomb
```

Internally, CoQuí computes the THC Coulomb integrals via the ISDF algorithm: 

$$
\phi^{\mathbf{k}}_{a}(\mathbf{r}) \longrightarrow \left\{ X^{\mathbf{k}}_{\mu a}, V^{\mathbf{q}}_{\mu \nu} \right\}
$$

This occurs either during construction (if `"init": True`) or later by calling `thc.init()` (if `"init": False`). 

Once initialized, the `ThcCoulomb` object serves as a handler for the Coulomb Hamiltonian, and is passed to all electronic structure methods in CoQuí, e.g:
```python
coqui.run_hf(thc, hf_params)
```

> 💡 **Note:** 💡
> - A full list of parameters for `thc_params` can be viewed using `help(coqui.make_thc_coulomb)`. 
> - Access to the internal THC tensors in `ThcCoulomb` is currently available only at the C++ level. Direct access from Python is not yet supported.

In [2]:
help(coqui.make_thc_coulomb)

Help on function make_thc_coulomb in module coqui.interaction.thc:

make_thc_coulomb(mf, eri_params)
    Create a THC Coulomb interaction handler using interpolative separable density fitting (ISDF).
    
    Parameters
    ----------
    mf : coqui Mf class
      The mean-field handler for the target system.
    
    eri_params: dict
      Parameters for the THC Coulomb interaction. Supported keys include:
    
        - init: bool, default=True
          If True, initializes the THC computation at construction.
          If False, defer initialization until `.init()` is called.
    
        - nIpts: int
          Number of THC interpolation points.
          Acts as one of the stopping criteria for the THC decomposition.
    
        - thresh: float
          Threshold for the THC decomposition.
          Also acts as a stopping criterion.
    
          Note: If both `nIpts` and `thresh` are provided, the THC algorithm terminates when either condition is met.
    
        - ecut: fl

### 🧪 Exercise 1: Compute your first THC Coulomb integrals

- Copy and paste the above THC python script
- Modify `thc_params` to postpone THC initialization during construction. You should get `False` from `thc.initialized()`.
- How many THC auxiliary basis you got from CoQuí? What is the $N_{\mu}/N_{\mathrm{orb}}$ ratio?

In [3]:
coqui.set_verbosity(mpi, output_level=2, debug_level=0)
# Mf for the target system
mf_params = {
    "prefix": "si", 
    "outdir": "../data/qe_inputs/si_222/out", 
    "filetype": "h5",
    "nbnd": 10
}
mf = coqui.make_mf(mpi, mf_params, "qe")

# Construct ThcCoulomb and compute the THC integrals during initialization
thc_params = {
    "init": False,
    "storage": "incore",
    "ecut": 35,
    "thresh": 1e-2
}
thc = coqui.make_thc_coulomb(mf, thc_params)
thc.init()

  Brillouin zone symmetry info
  ----------------------------
  Q-points in the irreducible zone = 4
  Symmetries applied to Q-points   = 3
  Time-reversal k-point pairs      = 0

  Quantum ESPRESSO reader
  -----------------------
  Number of spins                = 1
  Number of polarizations        = 1
  Number of bands                = 10
  Monkhorst-Pack mesh            = (2,2,2)
  K-points                       = 8 total, 4 in the IBZ
  Number of electrons            = 8
  Electron density energy cutoff = 200.000 a.u. | FFT mesh = (36,36,36)
  Wavefunction energy cutoff     = 28.496 a.u. | FFT mesh = (17,17,17), Number of PWs = 1989

  Brillouin zone symmetry info
  ----------------------------
  Q-points in the irreducible zone = 4
  Symmetries applied to Q-points   = 3
  Time-reversal k-point pairs      = 0

  Quantum ESPRESSO reader
  -----------------------
  Number of spins                = 1
  Number of polarizations        = 1
  Number of bands                = 10
  Monkhor

### 🧪 Exercise 2: Save the THC Coulomb integrals 

While THC decomposition offers significant computational savings in many-body simulations, the construction of the THC Coulomb Hamiltonian itself can be as costly as the downstream calculations. Fortunately, these integrals can be reused across different simulations, making it highly beneficial to store them for future use. 

Your tasks: 
- Modify `thc_params` to instruct CoQui to save the THC integrals to an HDF5 file using the `"save"` option.
- Use the command-line tool `h5ls` to inspect the contents of the resulting HDF5 file.
- Try to connect the stored datasets to the key quantities described in the ISDF algorithm (e.g., collocation matrices, auxiliary basis, projected Coulomb kernel).
- Create a new `ThcCoulomb` by reading the precomputed THC HDF5 file. 

In [3]:
# Save the THC integrals to "thc.coulomb.h5"
thc_params["save"] = "thc.coulomb.h5"
thc_save = coqui.make_thc_coulomb(mf, thc_params)
thc_save.init()

  Electron-electron interaction kernel
  ------------------------------------
  type          = coulomb
  ndim          = 3
  cutoff        = 1e-08
  screen_type   = none


╔═╗╔═╗╔═╗ ╦ ╦╦  ╔╦╗┬ ┬┌─┐╔═╗┌─┐┬ ┬┬  ┌─┐┌┬┐┌┐ 
║  ║ ║║═╬╗║ ║║   ║ ├─┤│  ║  │ ││ ││  │ ││││├┴┐
╚═╝╚═╝╚═╝╚╚═╝╩   ╩ ┴ ┴└─┘╚═╝└─┘└─┘┴─┘└─┘┴ ┴└─┘

  Algorithm                       = ISDF
  THC integrals access            = incore
  Found precomputed THC integrals = false
  --> CoQuí will compute THC integrals and save to: thc.coulomb.h5

  ERI::thc Computation Details
  ----------------------------
  Energy cutoff                = 35.0 a.u. | FFT mesh = (19,19,19), Number of PWs = 2685
  Default Slate block size     = 1024
  Default cholesky block size  = 8
  Threshold                    = 0.01
  Distribution tolerance       = 0.2
  Fraction of memory used for estimation = 0.75
  --> CPU Memory Available: 457810 
 

*******************************
 ERI::thc::interpolating_points 
*******************************
  -memor

In [4]:
# Read the precomputed THC integrals 
thc_read = coqui.make_thc_coulomb(mf, thc_params)
thc_read.init()


╔═╗╔═╗╔═╗ ╦ ╦╦  ╔╦╗┬ ┬┌─┐╔═╗┌─┐┬ ┬┬  ┌─┐┌┬┐┌┐ 
║  ║ ║║═╬╗║ ║║   ║ ├─┤│  ║  │ ││ ││  │ ││││├┴┐
╚═╝╚═╝╚═╝╚╚═╝╩   ╩ ┴ ┴└─┘╚═╝└─┘└─┘┴─┘└─┘┴ ┴└─┘

  Algorithm                       = ISDF
  THC integrals access            = incore
  Found precomputed THC integrals = true
  --> Reading the precomputed THC integrals from file: thc.coulomb.h5

  Summary of THC Coulomb Integrals
  --------------------------------
  Number of interpolating points = 97
  X orbital index range          = [0,10)
  Y orbital index range          = [0,10)

####### End of THC initialization routines #######



### 🧪 Exercise 3: Convergence of the THC decomposotion

1. Modifying the following param dictionary to systematically improve the accuracy of THC decomposition.
2. The THC accuracy is estimated using the utility function `check_thc_accuracy({your.thc.h5})`
3. Plot the THC accuracy v.s. number of interpolating points. What do you see? 

### Miscellaneous contents 

## 1.2 Read THC Coulomb Hamiltonian from a precomputed HDF5

The computed THC Coulomb integrals are accessed by passing the `[interaction]` to subsequent many-body
calculations (see [mbpt](../mbpt/README.md)). Optionally, the THC integrals can be saved to
an HDF5 file with the following data structure:
```text
group      /
dataset    /Np                       # Size of the THC auxiliary basis (N_mu)
dataset    /collocation_matrix       # Collocation matrix X^k
dataset    /coulomb_matrix           # Coulomb matrix in the THC auxiliary basis V^q
```
The HDF5 checkpoint file for THC Coulomb integrals is particularly useful for reusing the Coulomb integrals
in various calculations and for external developments in electronic structure theories (see
[thc_eri.toml](thc_eri.toml) for more details).

The size of the THC auxiliary basis ($N_{\mu}$) can either be set explicitly by the user or determined automatically to meet a specified accuracy. In practice, $N_\mu = O(N_{\mathrm{orb}})$ is typically sufficient due to the low-rank structure of the full Coulomb tensor. 
